In [1]:
import tensorflow
tensorflow.__version__

'2.3.0'

In [7]:
from tensorflow.keras.applications import inception_v3
from tensorflow.keras import backend as K
import tensorflow as tf

tf.compat.v1.disable_eager_execution()

In [8]:
K.set_learning_phase(0)
model = inception_v3.InceptionV3(weights='imagenet',include_top=False)

In [9]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, None, None, 3 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, None, None, 3 0           batch_normalization_94[0][0]     
_______________________________________________________________________________________

In [10]:
layer_contributions = {
'mixed2': 0.2,
'mixed3': 3.,
'mixed4': 2.,
'mixed5': 1.5,
}

In [11]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss = loss + coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

In [12]:
dream = model.input
grads = K.gradients(loss, dream)[0]
grads/= K.maximum(K.mean(K.abs(grads)), 1e-7)
outputs = [loss, grads]

fetch_loss_and_grads = K.function([dream], outputs)

In [13]:
def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

In [14]:
def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...Loss value at', i, ':', loss_value)
        x += step * grad_values
    return x

In [15]:
import scipy
from keras.preprocessing import image

In [16]:
def resize_img(img, size):
    img = np.copy(img)
    factors = (1,float(size[0]) / img.shape[1],
    float(size[1]) / img.shape[2],1)
    return scipy.ndimage.zoom(img, factors, order=1)

In [17]:
import imageio
def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    imageio.imwrite(fname, pil_img)
    #scipy.misc.imsave(fname, pil_img)

In [18]:
def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

In [19]:
def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [21]:
import numpy as np
step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20
max_loss = 10.
base_image_path = 'pic.jpg'
img = preprocess_image(base_image_path)


In [22]:
original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

In [23]:
successive_shapes = successive_shapes[::-1]

In [24]:
original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

In [25]:
for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,iterations=iterations,step=step,max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img
    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')
save_img(img, fname='final_dream.png')

Processing image shape (783, 551)
...Loss value at 0 : 1.7984906
...Loss value at 1 : 2.3408027
...Loss value at 2 : 3.055122
...Loss value at 3 : 3.9152904
...Loss value at 4 : 4.8112636
...Loss value at 5 : 5.6941357
...Loss value at 6 : 6.5432644
...Loss value at 7 : 7.37998
...Loss value at 8 : 8.163201
...Loss value at 9 : 8.932549
...Loss value at 10 : 9.701781
Processing image shape (1097, 771)
...Loss value at 0 : 3.602104
...Loss value at 1 : 5.0161576
...Loss value at 2 : 6.1870527
...Loss value at 3 : 7.2251205
...Loss value at 4 : 8.229747
...Loss value at 5 : 9.168282
Processing image shape (1536, 1080)
...Loss value at 0 : 3.501968
...Loss value at 1 : 4.762478
...Loss value at 2 : 5.876562
...Loss value at 3 : 6.935107
...Loss value at 4 : 7.9819884
...Loss value at 5 : 9.002198
...Loss value at 6 : 9.995281
